In [101]:
import pandas as pd
import numpy as np
import numpy_financial as npf

In [1]:
from portfolio_analysis.data import get_single_company_data, get_all_company_tickers
from portfolio_analysis.dcf import get_irr, project_eps

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from portfolio_analysis.api import apikey

In [4]:
company_names = get_all_company_tickers(apikey)

In [5]:
company_names.head()

,symbol,name,price,exchange,exchangeShortName,type
0,SPY,SPDR S&P 500 ETF Trust,434.24,New York Stock Exchange Arca,AMEX,etf
1,CMCSA,Comcast Corporation,57.21,Nasdaq Global Select,NASDAQ,stock
2,KMI,"Kinder Morgan, Inc.",16.91,New York Stock Exchange,NYSE,stock
3,INTC,Intel Corporation,53.86,Nasdaq Global Select,NASDAQ,stock
4,MU,"Micron Technology, Inc.",70.99,Nasdaq Global Select,NASDAQ,stock


In [6]:
import ipywidgets as widgets

In [7]:
company_names[company_names.symbol == 'AAPL']

,symbol,name,price,exchange,exchangeShortName,type
10,AAPL,Apple Inc.,142.65,Nasdaq Global Select,NASDAQ,stock


In [10]:
exchange_name = widgets.Dropdown(
    options=company_names.exchange.sort_values().unique(),
    value='NASDAQ',
    disabled=False,
); exchange_name

Dropdown(index=32, options=('', 'AMEX', 'ASE', 'Amsterdam', 'Athens', 'BATS', 'BATS Exchange', 'Berlin', 'Brus…

In [11]:
company_name = widgets.Dropdown(
    options=company_names[company_names.exchange == exchange_name.value].name.sort_values().to_list(),
    disabled=False,
); company_name

Dropdown(options=(' Nuveen Corporate Income November 2021 Target Term Fund', ' Saba Capital Income & Opportuni…

In [ ]:
# Issues Identified -

# Bug: Was only taking mean of -3 value 

# Outlier EPS ROC for eg: eps change from 0.01 to 2 - huge roc OR changes from -ve to +ve
# Solution:
# Changed mean of prior 3 periods eps and eps_roc to median to reduce effect of outliers to get base eps and eps_roc
# Have a flag to indicate if eps_roc over the last 3 years has been only positive or if there was a -ve value
# List eps over the past 3 periods

# Negative eps_roc
# Resulted in + eps is alternate years
# Solution: Is eps_roc is less than 0.01 then change to 0.01

# Rolling median eps_roc is an outlier
# Solution: take the lower of the eps_roc and the rolling_rps_roc

In [155]:
company = company_names[(company_names.name == company_name.value) & (company_names.exchange == exchange_name.value)].iloc[0]
company_data_dict = get_single_company_data(company["symbol"], apikey)
IS = company_data_dict["IS"]
profile = company_data_dict["Profile"]
BS = company_data_dict["BS"]
MC = company_data_dict["MC"]
CFR = company_data_dict["CFR"]
company_analysis = get_irr(company, 
                       company_data_dict["IS"], 
                       company_data_dict["Profile"],
                       company_data_dict["BS"],
                       company_data_dict["MC"],
                       company_data_dict["CFR"])

/Users/murtazavahanvaty/OneDrive - Cornerstone Capabilities/Personal/portfolio-analysis/portfolio_analysis/dcf.py:317: RuntimeWarning: invalid value encountered in double_scalars
  else:


In [156]:
company_analysis

symbol                                         ITCB
name                                 Itaú Corpbanca
price                                          3.71
irr                                      101.282237
npv                                     2505.092796
eps_roc                                        0.01
eps_diluted                                0.296016
MOP                                             0.0
QA                                              NaN
MCap                                  1267354520.32
PE                                         1.278541
eps_roc_flag                               Negative
eps_base                                      372.0
eps_list         [504.0, 372.0, -472.6372081286305]
error_message                                      
dtype: object

In [140]:
df = IS
df["year"] = df["date"].apply(lambda x: x[:4])
#     Find rate of change of eps yoy
df = df.sort_values(by="year")
df[f"{metric}_roc"] = df[f"{metric}"].pct_change()

In [154]:
df[['year','eps','eps_roc']]

,year,eps,eps_roc
20,2000,135.000135,NaN
19,2001,270.000270,NaN
18,2002,255.000255,NaN
17,2003,390.000390,0.529412
16,2004,330.000333,-0.055556
15,2005,346.807763,0.050932
14,2006,254.792586,-0.153846
13,2007,330.179614,0.050932
12,2008,372.241634,0.127391
11,2009,577.500578,0.295876


In [151]:
df[f"{metric}_roc"] = df[f"{metric}_roc"].rolling(3).median()

In [152]:
eps_base = df[f"{metric}"].iloc[-3:].median()
eps_roc = df[f"{metric}_roc"].iloc[-3:].median()

In [153]:
eps_roc

1.5454545454545454

In [143]:
new_df = pd.DataFrame()
new_df["year"] = [i for i in range(int(df.loc[0, 'year']) +1, int(df.loc[0, 'year']) +11)]
future_eps = []
#     DO this for a optimistic and pessimistic scenario
for i in range(len(new_df["year"])):
    if i == 0:
        future_eps.append(eps_base * (1 + eps_roc))
    else:
        future_eps.append(future_eps[i - 1] * (1 + eps_roc))
new_df[f"{metric}"] = future_eps

In [144]:
new_df

,year,eps
0,2021,274.571429
1,2022,202.659864
2,2023,149.582281
3,2024,110.405969
4,2025,81.490120
5,2026,60.147469
6,2027,44.394561
7,2028,32.767414
8,2029,24.185472
9,2030,17.851182
